# Week 2: decoding analyses

This week's tutorial is about how to implement decoding analyses in Python. We'll be looking at the implementation of concepts like K-fold cross-validation, feature-selection/extraction, model fitting/prediction, permutation testing, and feature visualization. To do so, we'll use the [`scikit-learn`](http://scikit-learn.org) machine learning package in Python (the de-facto and most-used ML package in Python). In this tutorial, we'll use the PIOP-dataset (**P**opulation **I**maging **O**f **P**sychology), a large-scale (>240 subjects) multimodal MRI-dataset, including structural (gray-matter [VBM](https://en.wikipedia.org/wiki/Voxel-based_morphometry) and white-matter [TBSS](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/TBSS)), resting-state (incl. [dual regression](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/DualRegression) data), and task-based functional data. Along with this set of MRI-scans, the PIOP-study also acquired a set of behavioral (psychometric) variables from various questionnaires and behavioral task (outside the scanner) of the same participants who were scanned. As such, this makes it an excellent data-set for decoding analyses in which you can use neural patterns (from any MRI modality; `X`) to predict a behavioral variable (`y`). This is, by the way, also one of the data-sets that you can use for your final project (more info on Blackboard/week_4).

For this tutorial, we'll focus on a specific decoding analysis in which *we try to predict intelligence (y) from brain patterns (X) associated with working memory processing* (based on the same data as you practiced with in the tutorial of week 1). Throughout the tutorial, we'll discuss all the steps necessary to answer this research question. (Mini-ToThink: is this a within- or between-subject analysis?)

In terms of skills, after this tutorial you are be able to:

* Load and appropriately transform dependent variables (`y`) corresponding to your patterns (`X`);
* Apply basic preprocessing to your patterns;
* Implement [feature-selection](http://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) and extraction processes;
* Build fully cross-validated [pipelines](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) using scikit-learn;
* Run appropriate statistics (including permutation tests) on your model performance scores;

Notably, we do *not* cover feature visualization because it is quite complicated to implement, especially in combination with feature-selection. Also, "brain mapping" should not be your goal in decoding studies, anyway. 

That said, in combination with the knowledge/skills you acquired from last week's tutorial, you'll be able to implement a complete decoding pipeline yourself at the end of this tutorial!

### Names
student 1: fill in your name ...

student 2: ... and the name of the person you're working with

## 1. Data representation (yet again)
Before we'll start with the machine learning stuff, we have to load in our patterns again. Like last week, you are going to use a custom `Mvp` class for this again, which we'll extend throughout this tutorial. You may use your own class from last week (given that it contains correct implementations of the load, standardize and apply_mask methods), or use the one in the solutions-notebook from last week. In any case, copy the code for the Mvp-class below:

In [ ]:
import numpy as np
import nibabel as nib

# Copy here below:


Now, given our research question, we're going to use the (between-subject!\*) patterns of the working-memory task again. More specifically, we are going to use the patterns from the "active-passive" contrast of each subject (just like last week).

---------------
\* It's a between-subject analysis because intelligence is a factor that varies at the subject level!

What we'd like to do now is to load in the tstat3 patterns from all subjects who completed the working-memory task in the scanner. We put this dataset in the `/home/Public/FirstLevel_piop` folder. This folder (and all its subdirectories) is accessible (read-only, so you cannot modify anything) for all NIPA-students. 

<div class='alert alert-warning'>
**ToDo**: Click on the "Files" icon on your desktop and navigate (just by clicking, not in the terminal) to this directory (note: you have to go up one directory relative to your `home` directory).<br><br>

Check out the structure. You should see a bunch of subdirectories with subject-names (e.g. pi0010, pi0204, pi0084). These directories contain the first-level data (.feat directories) from the different task-based MRI-runs and some separate nifti-files (we'll get to these later). As you can see, we already estimated the (between-subject) patterns for you (which can be found in the .feat directories)! Go to a random subject's directory and click on the `*piopwm.feat` directory (note that not *all* subjects have this directory, because some subjects did not complete this run in the scanner due to technical issues or time issues). Check out the `design.png` file and the `reg_standard` directory (which files are in there? Do you understand what each file represents?). 
</div>

As you can see, the data is structured in an hierarchical way:

`/home/Public/FirstLevel_piop/{subject-no}/{task}.feat/reg_standard/{tstat-files}.nii.gz`

Given this structure, it's easy to `glob` a set of tstat-files for a specific task across all subjects. 

<div class='alert alert-warning'>
**ToDo**: use glob to find all the paths to the tstat3-files of the piopwm task. (Hint 1, you will need *two* wildcards in your search-string; Hint 2, you should find 217 paths in total)
</div>

In [ ]:
# Implement the todo here (insert the right search-query in glob()):
from glob import glob

paths = glob(FILL IN YOUR SEARCH STRING HERE)

Let's sort the paths, such that the paths are sorted by subject-number (pi0010 to pi0248). Also, let's not use *all* data (all subjects) at once, because that's actually quite memory-intensive and we don't want the TUX-server to crash. Therefore, we'll select only every third path (i.e. path 1, 4, 7, 10, etc.). 

In [ ]:
paths = sorted(paths)
print("Number of paths was: %i" % len(paths))
subset_paths = paths[1::3]
print("Number of paths is now: %i" % len(subset_paths))

Alright, now we have 72 paths (sorted by subject-number) to the subjects' tstat3 files. Let's initialize the Mvp object and call the `load()` method! 

<div class='alert alert-info'>
**ToThink**: before you initialize the Mvp object and load the patterns, can you predict what shape the `X`-attribute of the Mvp-object will be? (Hint: images in MNI152 2mm space are of shape 91\*109\*91)
</div>

Alright, less thinking, more doing:

In [ ]:
mvp = Mvp(paths=subset_paths)
mvp.load()

Let's check the shape of the `X`-attribute:

In [ ]:
print("Shape of X is: %s" % (mvp.X.shape,))

### A note about standardization
We briefly touched upon the issue of "standardization" last week, but the process of standardization is especially important in decoding analyses, so it warrants some extra explanation. 

As explained before, standardization refers to the process of converting numbers in a vector to the (signed) number of standard deviations it deviates from the mean of the vector. Formally, for a vector `x` with mean $\bar{x}$ and standard deviation $s_{x}$ and length `N`, the standardized value of element $x_{i}$ is:

\begin{align}
x_{i} = \frac{x_{i} - \bar{x}}{s_{x}}
\end{align}

Practically, standardization in machine learning is applied to each column (i.e. feature), ensuring that each feature (in decoding: voxel) is on the same scale. In other words, standardization ensures that each feature has a mean of 0 and a standard deviation of 1.

"But *why* do you have to standardize your features?", you might reasonably ask. Well, that has to do with the way many ML models calculate their parameters (i.e. $\beta$). For most models, this process may fail to converge and thus is unable to calculate the model's parameters if not all features are on the same scale. 

Anyway, just remember that *before you do any model fitting* your features should be standardized\*. Fortunately, you already implemented this yourself last week as a method in your mvp-class: the `standardize()` method! So, before we go on, let's just standardize our features:

-----------
\* There are also other ways to get your features on the same scale, like min-max scaling, but standardization is by far the most often used method.

In [ ]:
# This will give a warning (invalid value in divide) because some voxels are zero across all samples (voxels outside the brain!),
# which will cause us to divide 0 by 0 for those voxels. You can simply ignore the warning!
mvp.standardize()

Cool, we got the (standardized) patterns (`X`) that we wanted. But as you might have noticed, we're still missing one crucial component for our decoding analysis: `y` (our to-be-decoded feature)! For our research question, we chose to focus on decoding intelligence, but how do we load and represent this in our pipeline? This is what the next section is about!

## 2. Adding a dependent variable (`y`)
While there is kind of a generic way to load in voxel patterns, there is usually not a single way to load in your dependent variable (y), because the exact factor that represents `y` dependent on your exact research question (and also depends how you have stored this data on disk). 

We'll focus on extracting a DV from a common tsv-file (like a csv-file, but not with comma- but tab-delimited values). Before we do that, just open the file and take a look at its structure. Just go to your desktop, double-click "Files", go up one directory (to `/home`) and go to `/home/Public/FirstLevel_piop`. Scroll aaaall the way down (past all the pi\* folders) and you'll find a file called `PIOP_BEHAV_NIPA.tsv` file. Double-click this, and press "Display". A "Text Import" window will pop up - select here "Tab" under the "Separated by" option. Then, click "OK". 

You'll see a spreadsheet with the subject-names in the first column and different variables in the columns. Basically, you can choose any variable in the columns as your to-be-decoded factor! For our example, we will pick the 'Raven' variable, which represents the subjects' (continuous) scores on the [Raven](https://en.wikipedia.org/wiki/Raven%27s_Progressive_Matrices) test, which aims to measure fluid intelligence. 

### 2.1. Loading behavioral data
To load in the tsv-file, we'll use `pandas`, a Python package for working with tabular data. It's main data structure is called the `DataFrame`, which is very similar to the dataframe-object in R. (Working with pandas is beyond the scope of this course, but you'll see it now and then throughout the course when we work with tsv-files.)

We'll walk you through how this is done:

In [ ]:
import pandas as pd # This is how pandas usually is imported

path_to_tsv = '/home/Public/FirstLevel_piop/PIOP_BEHAV_NIPA.tsv'

# the read_csv function is the standard way to load in spreadsheet-like data
df = pd.read_csv(path_to_tsv, sep='\t', index_col=0)

# Let's see how this looks like
df.head()

As you can see above, the DataFrame is simply a spreadsheet-like representation of the data, with the subjects represented as different rows and the different behavioral/demographic variables represented as columns. Now, we'd like to extract the `Raven_raw` columns, which represents the raw (summed) scores on the Raven-test for each subject. To do so, we can simply index the `df` variable with the string "Raven_raw" (just like indexing a dictionary). 

In [ ]:
y = df['Raven_raw']
print("Length of y is: %i\n" % y.shape[0])
print("y looks like:\n\n%r" % y)

`y` is now a subset of the original dataframe (df) that only contains the Raven scores and a corresponding "index", which refers to the subject it belongs to. 

But we have a problem! 

`y` now contains 248 values (from 248 subjects), but our X attribute contains patterns from only 72 subjects (the subset selected earlier). For our analysis, the samples in our patterns (X) should match the entries in our dependent variable (y). How do we do this? Well, we need to extract which subjects were included in X and then use those subjects to "index" y. 

Below, we show you a way to do this.

In [ ]:
# First we need to extract which subjects were actually included in X.
# We do this by extracting this info from the "subset_paths" from earlier
# using a list-comprehension (a fancy for loop)
subject_names = [path.split('/')[4] for path in subset_paths]

print(subject_names)

In [ ]:
# Here we use the pandas dataframe "loc" method, which slices the DataFrame by their
# index names (and we subsequently transform the result to a numpy array)
y_new = np.array(y.loc[subject_names])

# And we can see now that the length of y is the same as the number of samples in X
print(mvp.X.shape)
print(y_new.shape)

Great! Now we have a set of patterns (X) with a corresponding vector representing the to-be-decoded feature (y). There is, however, still one issue worth discussing. At this moment, `y` is a continuous measure (sum-score of performance on the Raven test). It follows then that our decoding analysis would use regression, but we will actually recode `y` such that it is categorical, as explained in the next section. 

### 2.2. Continuous vs. categorical DVs
In some scenario's you might want to recode your continuous dependent variable (y) into a categorical one. One reason is simply because fMRI patterns are incredibly noisy, and classification analyses are more robust than regression analyses. Note, though, that if you recode your continuous DV into a categorical one, your conclusions about your model also become weaker: regression models allow you to explain/model variance across a larger interval than classification models.

For example, would you be more impressed if told you I could predict your IQ with an error of +/- 2 points (a continuous DV) or if I told you I could only predict whether your IQ is higher or lower than 100 (a categorical DV)? Given that your dependent variable is truly continuous, it's better to subject it to a regression analysis in terms of strength of your conclusion, but given the very noisy nature of fMRI data it may be advantageous to trade in some of this inferential strength for some extra SNR by recoding your DV into a categorical variable. 

This is probably also the reason that *classification* analyses are way more popular in the fMRI community than *regression* analyses. That's why we'll recode the Raven score into a categorical variable and continue with that for the rest of the tutorial. 

But how should we recode our DV? There are various ways to do this, but we'll use the median-split: any values above the median of `y` will be coded as 1 and any values below the median of `y` will be coded 0. Thus, our updated decoding goal is to predict whether someone has a high intelligence (`y = 1`) or low intelligence (`y = 0`).  

We'll show you how to implement this median-split transfor below ... No wait, you know what, by now you guys should be quite the Python experts: try it yourself!

<div class='alert alert-warning'>
**ToDo**: Transform mvp.y into categorical values {0, 1} using a median-split (you can use the numpy-function `median` to do this, i.e. np.median(some_array)). Hint: maybe you can use the [astype()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.astype.html) method of numpy arrays to convert boolean arrays to integer arrays?  
</div>

In [32]:
# Implement the median-split here!


Okay, finally, we have everything we need for a decoding analysis: a sample-by-voxel array (X) representing the patterns and a vector with the corresponding to-be-decoded (categorical) variable (y). This is really all we need for a decoding analysis, we promise.

But the process of loading in a y-variable, making sure it corresponds to the same subjects contained in X, and transforming it using a median-split is kinda messy. For the following interim assignment, you have to incorporate this process in your Mvp class! 

<div class='alert alert-danger'>
**Assignment 1** (6 points): In this assigment, you will extend your Mvp class such that it is able to load in a y-variable from a tsv-file and index it such that it contains the data from the same subjects that were included in the X-attribute. So, first copy-paste the code for your class in the code-cell below. Then, do three things:<br><br>

1. *Within* your load() method, extract the subject-names from the `paths` attribute and store the results (which should be a list with subject-names, like this: ['pi0010', 'pi0011', 'pi0012', ..., etc.]) in a *new* attribute named 'subject_names'. You can use the list-comprehension defined earlier if you want to extract the subject-names. So, after calling the load() method, your mvp-object should also contain the attribute `subject_names` which represents which subjects the X-attribute is based upon.<br><br>

2. Then, add a new method named `load_y` that takes two arguments (not counting *self*): a path (a string) to a tsv-file and a variable name (a string) referring to the column-name that should be extracted from the loaded tsv-file. Then, your method should load the tsv-file using pandas and extract the correct column. Also, it should filter the resulting values with the attribute `subject_names` such that you only extract y-values for the subjects that are actually included in X (using the `loc` method). The resulting (filtered) y-values should be stored *as a numpy array* in a *new* attribute called `y`. So, after calling `load_y`, mvp should contain the attribute `y`.<br><br>

So, calling the method would look something like this:<br> `my_mvp.load_y(path_to_tsv='/some/path/to/tsv-file.tsv', variable='Raven_raw')`<br>
Then, after calling `load_y`, the attribute `y` should be accessible (i.e. mvp.y should exist).<br><br>

3. Add another method, named `median_split_y` that takes no arguments (apart from *self*) and transforms the internal `y`-attribute into only 0 and 1 values using a median split.<br><br>

In the code-cell below your copy-pasted code for your Mvp class, we included some code to check whether your extension of Mvp is correct. If this cell runs without errors, you completed the assignment correctly.
</div>

In [40]:
# Copy your Mvp class definition below and extend it 


### Test your answer!
Run the cells below to see if your extensions are correct! If you get no errors, your code is correct!

In [ ]:
# Test of the load() method
mvp = Mvp(subset_paths)
mvp.load()

# Check if X-attribute exists
assert(hasattr(mvp, 'X'))

# Check if subject_names attribute exists
assert(hasattr(mvp, 'subject_names'))

# Check if the length of subject_names is the same as the rows in X
assert(len(mvp.subject_names) == mvp.X.shape[0])

print('You extended the load() method correctly!')

In [ ]:
# First, we execute some code to see whether the class changed (e.g. if you added a method)
# because, if so, we need to recreate the mvp class (we need to update it to the newest class!)

# if it changed ...
if Mvp != mvp.__class__:
    # ... then re-initialize it
    mvp = Mvp(subset_paths)
    mvp.load()

# Now, test load_y
mvp.load_y(path_to_tsv, 'Raven_raw')

# Check if y-attribute exists
assert(hasattr(mvp, 'y'))

# Check if y is indeed a numpy array
assert(isinstance(mvp.y, np.ndarray))

# Check if y is correctly filtered (i.e. only the subjects that are in subject_names)
assert(len(mvp.y) == len(mvp.subject_names) == mvp.X.shape[0])

print('You implemented the load_y() method correctly!')

In [ ]:
# First, check if it changed ...
if Mvp != mvp.__class__:
    # ... if so, re-initialize it
    mvp = Mvp(subset_paths)
    mvp.load()
    mvp.load_y(path_to_tsv, 'Raven_raw')

# Test the median_split_y() method
mvp.median_split_y()

# Check if y is now binary {0, 1}
assert(all(np.unique(mvp.y) == [0, 1]))

# Check if nothing weird happened to the length of y
assert(len(mvp.y) == mvp.X.shape[0])

print('You implemented the median_split_y() method correctly!')

## What's next ...
Awesome, we now have everything we need (X and y) packaged neatly in an instance of your Mvp-class! As might have become clear by now, using classes to "package"/organize code can be quite efficient, especially if you're modifying variables a lot (like preprocessing of X). 

In the rest of the tutorial, you'll finally learn how to actually implement decoding pipelines. As discussed in the lecture, typical pipelines consist of the following elements:

* Data partitioning
* Feature selection/extraction
* Model fitting (on train-set)
* Model cross-validation (on test-set)
* Calculate model performance
* Statistical analyses of performance
* Optional: feature visualization 

In the rest of the tutorial, we'll discuss these topics in a slightly different order. First, we'll discuss how you can fit and cross-validate scikit-learn models, and while we're at it, how to calculate model performance (here: accuracy). Subsequently, you'll learn how to embed these concepts in fully cross-validated K-fold data partitioning schemes. Then, we'll extend our pipelines with feature selection methods. Finally, we'll show you how to implement a permutation test and briefly show you how to visualize features. 

## 3. Model fitting & cross-validation
First, we'll show you how to use scikit-learn models. In fact, the most useful functionality of scikit-learn is probably that they made fitting and cross-validating models (or, in scikit-learn lingo: estimators) trivially easy. 

Note that in the upcoming example, we are going to fit the model on *all* our samples - this is something you'd never do of course (you always need to cross-validate it to a new, independent sample). 

Anyway, let's import a scikit-learn model: the (linear) support vector classifier ([SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)), which is one of the most-often used models in fMRI pattern analyses. In scikit-learn, this model is part of the (suprising...) `svm` module:

In [ ]:
# Scikit-learn is always imported as 'sklearn'
from sklearn.svm import SVC

Like most of scikit-learn's functionality, SVC is a *class*. So, let's initialize an SVC-object! One important argument that this object needs is the ["kernel"](http://scikit-learn.org/stable/auto_examples/svm/plot_svm_kernels.html) you want the model to have. Basically, the kernel determines how to treat/process your features: linearly, or non-linearly (such as the `kernel='rbf'` or `kernel='poly'` options). As discussed in the lecture, most often a linear kernel is the best option (as non-linear kernels seem to overfit very quicly). 

To initialize an SVC-object with a linear kernel, just do as follows:

In [ ]:
# clf = CLassiFier
clf = SVC(kernel='linear')

Alright, so now we have initialized an SVC-object with a linear kernel. Now, you need to do two thing to get the prediction for each sample (i.e. whether they're predicted as 0 or 1): fit, using the method `fit(X, y)`, and predict the class (i.e. 0 or 1) for each class using the method `predict(X)`. Basically, in the `fit` method, the parameters of the model (i.e. $\beta$) are calculated. Then, in the `predict` method, the parameters are used to generate a prediction for each sample (i.e. 0 or 1). 

Let's first look at the `fit` method. As you can see, the `fit(X, y)` method with two parameters: X (a samples-by-features matrix) and y (a vector of length n-samples). Let's do that for our data stored in our mvp-object: 

In [ ]:
clf.fit(mvp.X, mvp.y)
print('Fitted SVC ...')

After calling the fit() method, the clf-object contains an attribute `coef_` that represent the model's parameters ('coefficients' in scikit-learn lingo, i.e. $\beta$). Let's check that out:

In [ ]:
coefs = clf.coef_
print("Shape of coefficients: %r" % (coefs.shape,))

Ah, just like we expected: the `coef_` attribute is exactly the same size as the number of voxels in our X-matrix (i.e. 91\*109\*91). 

Anyway, usually, you don't do anything directly with the weights (perhaps only if you want to do anything with feature visualization): scikit-learn handles that for you. What you *do* want, is an actual prediction ($\hat{y}$) for our samples!

To get this, simply call the `predict(X)` method of the model, which returns the predictions as an array:

In [ ]:
y_hat = clf.predict(mvp.X)
print("The predictions for my samples are:\n %r" % y_hat)

A logical next step is to assess how good the model was in predicting the class of the samples. A straightforward metric\* to summarize performance is *accuracy* which can be defined as: 

\begin{align}
accuracy = \frac{number\ of\ correct\ predictions}{number\ of\ predictions}
\end{align}

---------------
\* There are waaaay [more metrics to summarize model performance](http://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/) for classification models, such as precision, recall, F1-score, and ROC-AUC. These metrics are more appropriate than accuracy when you have *imbalanced classes*, i.e. more samples in one class (e.g. negative images) than in another class (e.g. positive images). Usually, however, experimental designs in fMRI pattern analyses are (more of less) balanced, so often you can just use accuracy. 

<div class='alert alert-warning'>
**ToDo**: Can you calculate the accuracy of the above model? Hint 1: you need to compare the true labels (i.e. mvp.y) with the predicted labels (i.e. y_hat). Hint 2: if you do arithmetic with boolean values (i.e. `True` and `False`), `True` is interpreted as 1 and `False` is interpreted as 0.
</div>

In [ ]:
# Implement your to-do here!
accuracy = 

If you've done the ToDo correctly above, you should have found out that the accuracy was 1.0 - a perfect score! "Awesome! Nature Neuroscience material!", you might think. But, as is almost always the case: if it's too good to be true, it probably *is* indeed too good to be true.

So, what is the issue here? Well, we didn't cross-validate the model! We fitted it on all the samples in the mvp-object and predicted the *same* samples, which leads to optimistic estimate of model performance. Such optimistic estimates in uncross-validated models are especially likely when there are many more features (here: voxels) than samples (here: subjects). In other words, we are probably *overfitting* the model here.

Thus, let's check what happens if we actually cross-validate the model. To do so, we need to partition the data into a train- and test-set. For this example, we'll use a simple hold-out scheme, in which we'll reserve half of the data for the test-set (we'll discuss more intricate cross-validation schemes such as K-fold in the next section).

Below, we index `X` and `y` such that the train-set will contain all odd-numbered samples and the test-set will contain all even-number samples:

In [ ]:
X_train = mvp.X[1::2]
print("Shape X_train: %r" % (X_train.shape,))

y_train = mvp.y[1::2]
print("Shape y_train: %r" % (y_train.shape,))

X_test = mvp.X[0::2]
print("Shape X_test: %r" % (X_test.shape,))

y_test = mvp.y[0::2]
print("Shape y_test: %r" % (y_test.shape,))

<div class='alert alert-warning'>
**ToDo**: now it's up to you to actually implement the *cross-validated* equivalent of the fit/predict procedure we showed you before. So, fit your model on `X_train` and `y_train` and then predict `X_test`. Calculate the cross-validated accuracy by comparing the predictions with `y_test`. 
</div>

In [ ]:
# Implement your ToDo here


Sweet! You actually performed your first proper decoding analysis in the ToDo above! There are however a couple of things we can do to improve the efficiency and results. One thing we can do is to use the data more efficiently in cross-validation. In the previous example, we split the data into two sets and fit the model on one (the train-set) and cross-validated to the other (the test-set). But as you've heard in the lecture, there is actually a way to "re-used" the data by using K-fold cross-validation, in which data is iteratively partitioned in train- and test-sets. This is explained in the next section.

## 3. Data partitioning: K-fold cross-validation
As discussed in the lecture, there are two principled ways of cross-validation: 

* Hold-out cross-validation;
* K-fold cross-validation;

In the previous section we implemented a form of hold-out cross-validation, which can be seen as a kind of "one-shot cross-validation". In fMRI data-sets, however, we usually have few samples (simply because MRI-data is expensive to acquire!), so K-fold cross-validation is more common than hold-out cross-validation in fMRI pattern analyses because it allows you to reuse the data.

Now, we can finally use some of scikit-learn's functionality. We are going to use the [StratifiedKFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) object from scikit-learn's model_selection module. Click the highlighted link above and read through the manual to see how it works.

Importantly, if you're dealing with a classification analysis, always use *Stratified*KFold (instead of the regular [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)), because this version makes sure that each fold contains the same proportion of the different classes (here: 0 and 1). Anyway, enough talking. Let's initialize a StratifiedKFold object with 5 folds:

In [ ]:
# scikit-learn is imported as 'sklearn'
from sklearn.model_selection import StratifiedKFold

# They call folds 'splits' in scikit-learn
skf = StratifiedKFold(n_splits=5)

Alright, we have a StratifiedKFold object now, but not yet any indices for our folds (i.e. indices to split X and y into different samples). To do that, we need to call the `split(X, y)` method:

In [ ]:
folds = skf.split(mvp.X, mvp.y)

Now, we created the variable `folds` which is officially a [generator](https://wiki.python.org/moin/Generators) object, but just think of it as a type of list (with indices) which is specialized for looping over it. Each entry in `folds` is a tuple with two elements: an array with train-indices and an array with test-indices. Let's demonstrate that\*:

-------------
\* Note that you can only run the cell below once. After running it, the `folds` generator object is "exhausted", and you'll need to call `skf.split(mvp.X, mvp.y)` again in the above cell.

In [ ]:
i = 1
for fold in folds:
    
    print("Processing fold %i" % i)
    # Here, we unpack fold (a tuple) to get the train- and test-indices
    train_idx = fold[0]
    test_idx = fold[1]
    
    print("Train-indices: %s" % train_idx)
    print("Test-indices: %s\n" % test_idx)
    
    i += 1
i = 0

As you can see, StratifiedKFold determined that for the first fold, sample 1 to 9 should be used for training and sample 0 and 2 (remember, Python uses 0-based indexing!) should be used for testing.

Now, we know how to access the train- and test-indices, but we haven't *actually* indexed our X and y (i.e. mvp.X and mvp.y) in the for-loop over folds. Nor have we actually fit the model on the train-set and cross-validated this to the test-set.

<div class='alert alert-warning'>
**ToDo**: in the code-cell below, complete the statements by indexing mvp.X and mvp.y to create four different objects in every fold: X_train, y_train, X_test, y_test. Also, we created a new classifier-object (clf) for you based on a different model: scikit-learn's `LogisticRegression` to show you that *every* model in scikit-learn works the same. Use this classifier to fit on the train-set and predict the test-set in every fold. Then, calculate the (cross-validated) accuracy in every fold. Keep track of the accuracies across folds, and after the loop over folds, calculate the average accuracy across folds.
</div>

In [ ]:
from sklearn.linear_model import LogisticRegression

# clf now is a logistic regression model
clf = LogisticRegression()
# run split() again to generate folds
folds = skf.split(mvp.X, mvp.y)

accuracy = []

for fold in folds:
    
    # This is a slightly more concise way to unpack tuples
    train_idx, test_idx = fold
    
    # Complete these statements by indexing mvp.X and mvp.y with train_idx and test_idx
    X_train = 
    y_train = 
    
    X_test = 
    y_test = 
    
    # ToDo: call fit (on train) and predict (on test)
    
    # ToDo: calculate accuracy

# ToDo: calculate average accuracy


Implementing K-fold cross-validation instead of hold-out cross-validation allowed us to make our analysis a little more efficient (by reusing samples). Another method that (usually) improves performance in decoding analyses is feature selection/extraction. 

## 4. Feature selection/extraction
As discussed before, a small sample/feature-ratio often results in overfitting (optimistic performance estimates on train-set). It follows then that reducing the amount of features often has a beneficial effect on cross-validated performance estimates.

Feature reduction can be achieved in two principled ways:

* feature extraction: transform your features into a set of lower-dimensional components;
* feature selection: select a subset of features

Examples of feature extraction are PCA (i.e. transform voxels to orthogonal components) and averaging features within brain regions from an atlas. Because feature extraction often does not use any information from the labels (`y`), this step does not need to be cross-validated (but you *can* cross-validate, e.g., PCA by estimating the components on the train-set only). 

Examples of feature selection are ROI-analysis (i.e. restricting your patterns to a specific ROI in the brain) and "univariate feature selection" (UFS). This latter method is an often-used data-driven method to select features based upon their univariate difference. 

Fortunately, scikit-learn has a bunch of feature selection/extraction objects for us to use. These objects ("transformers" in scikit-learn lingo) work similarly to estimators: they also have a `fit(X, y)` method, in which for example the univariate differences (in UFS) or PCA-components (in PCA-driven feature extraction) are computed. Then, instead of having a `predict(X)` method, transformers have a `transform(X)` method.

### 4.1. (Univariate) feature selection
As an example of feature selection, let's take a closer look at how UFS can be implemented using scikit-learn. Basically, it follows the following structure:

`transformer = Selector(score_func=ufs_function, other_args)`

Here, the `score_func` parameter takes a function that implements a univariate statistical test (like an [f-test](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif) or a [chi-square test](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif)). The `Selector` class determines how to select the subset of features based on the result of the `score_func`. 

For example, the [SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) selector selects the best `K` features based on their scores from the statistical test. For example, the following transformer would select the best 100 features based on their F-score:

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# f_classif is a scikit-learn specific implementation of the F-test
select100best = SelectKBest(score_func=f_classif, k=100)

Another example of a selector is `SelectFwe` which selects only features with statistics-values corresponding to p-values lower than a set alpha-level. For example, the following transformer would only select features with p-values from a chi-square test lower than 0.01: 

In [ ]:
from sklearn.feature_selection import SelectFwe, chi2
selectfwe_transformer = SelectFwe(score_func=chi2, alpha=0.01)

<div class='alert alert-info'>
**ToThink**: The functions `f_classif` and `chi2` are functions to calculate statistics for when the the DV is categorical (i.e. in classification scenarios). Can you think of a function that does the same when your DV is continuous (i.e. in regression scenarios)? 
</div>

But how does this work in practice? We'll show you an (not cross-validated!) example using the select100best transformer initialized earlier:

In [ ]:
# Fit the transformer ...
select100best.fit(mvp.X, mvp.y)

# ... which calculates the following attributes (.scores_ and .pvalues_)
# Let's check them out
scores, pvalues = select100best.scores_, select100best.pvalues_

# As you can see, each voxel gets its own score (in this case: an F-score)
print(scores.size)

We can also visualize these scores in brain space (it's a coronal slice sideways ...): 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

scores_3d = scores.reshape((91, 109, 91))
plt.imshow(scores_3d[:, 75, :], origin='lower', cmap='hot')
plt.axis('off')
plt.show()

Thus, in the image above, brighter (more yellow) colors represent voxels with higher scores on the univariate test. If we subsequently apply (or "transform" in scikit-learn lingo) our X-matrix using, for example, the `select100best` selector, we'll select only the 100 "most yellow" (i.e. highest F-scores) voxels.

<div class='alert alert-info'>
**ToThink**: Given the image above, what is the a major difference between data driven feature selection (like UFS) and ROI-based feature selection (e.g. only look at patterns in the amygdala) in terms of the spatial scale of patterns you'll select? Try to think of an example of UFS over ROI-based feature selection and vice versa.
</div>

Practically, after we've fit the transformer, we can call the `transform(X)` method that will actually select the subset according to the selector:

In [ ]:
print("Shape of mvp.X before transform: %r ..." % (mvp.X.shape,))
X_after_ufs = select100best.transform(mvp.X)

print("... and shape of mvp.X after transform: %r." % (X_after_ufs.shape,))

As you can see, the transformer correctly selected a subset of 100 voxels from our mvp.X matrix! Now, both selectors were fit on the entire dataset, which is often course not how it should be done: because they use information from the labels (`y`), this step should be cross-validated. Thus, what you have to do is:

* fit your transformer on the train-set;
* transform your train-set;
* transform your test-set;
* (fit your model on the train-set;)
* (cross-validate to the test-set;)

We summarized the entire pipeline, including data partitioning (indexing), feature selection (transformation), and model fitting and their corresponding cross-validation steps in the image below:
<img src='sklearn_transformers.png'>

### 4.2. Feature extraction
There are many methods for feature extraction, like "downsampling" to ROI-averages (i.e. averaging voxel patterns in brain regions) and dimensionality-reduction techniques like PCA. Scikit-learn provides some of these techniques as "transformer" objects, which again have a `fit()` and `transform` method. In the next ToDo, you'll apply this!

<div class='alert alert-warning'>
**ToDo**: Below, we import the `PCA` class from scikit-learn. Check out the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Now, initialize an object from this `PCA`-class with the number of components to store set to 5, and subsequently fit in on `mvp.X` and subsequently call transform on `mvp.X` (note that you do not have to cross-validate this procedure!) and store the result in a new variable named `X_pca_transformed`. Then, check out the shape of `X_pca_transformed`: does this make sense?
</div>

In [ ]:
from sklearn.decomposition import PCA

# initialize a PCA object ...


# ... call fit on mvp.X ...


# ... and call transform on mvp.X
X_pca_transformed = 

# And finally check out the shape of X_pca_transformed!


Now, for the assignment, you have to put everything you learned so far together and implement a fully cross-validated pipeline with UFS *and* model fitting/prediction!

<div class='alert alert-danger'>
**Assignment 2** (3 points): Below, we set up a K-fold cross-validation loop and prespecified a classifier (`clf`, a logistic regression model) and a transformer (`select50best`, selecting the 50 best features based upon an F-test). Now, it's up to you to actually implement the feature selection inside the for-loop. Make sure to fit the transformer only on the train-set, but then transform *both* the train-set and the test-set. Then, fit the model on the transformed train-set and cross-validate to the transformed test-set. Calculate accuracy of the cross-validated  model for each fold, and after all folds calculate the average accuracy (across folds).
</div>

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
folds = skf.split(mvp.X, mvp.y)

accuracy = []

select50best = SelectKBest(score_func=f_classif, k=50)

for fold in folds:
    
    train_idx, test_idx = fold
    
    # ToDo: make X_train, X_test, y_train, y_test
    
    # ToDo: call the select50best fit method (on train) and predict (on test)
    
    # ToDo: calculate accuracy

# ToDo: calculate average accuracy


### Optional: advanced cross-validation using Pipelines
(Recommended, but optional: only go through this section if you have time)

As you have seen in the previous assignment, the code within the K-fold for-loop becomes quite long (and messy) when you add a feature-selection step. Suppose you want to add another feature extraction step, like performing a PCA after an initial feature selection step. The code becomes even more obscure ...

Luckily, scikit-learn has an awesome solution for this: "Pipelines".

Pipelines are somewhat more advanced functionality within scikit-learn, but we wanted to show you guys because it really "cleans up" your code, and additionally, it is a great safeguard for accidental overfitting (e.g. when you accidentally perform feature-selection on all your data instead of only your train-data). 

Anyway, how does this work? Well, a picture it worth a thousand words, so check out the image below which schematically depicts what a pipeline does: <img src='pipelinesX.png'>

As you can see, a `Pipeline`-object "strings together" an arbitrary number of transformers (including "preprocessing" transformers like `StandardScaler`\*) and can optionally end in an estimator. Pipeline-objects have two (relevant) methods: `fit(X, y)` and `predict(X, y)` (the latter method only exists if there is an estimator in the pipeline). To use it, you only have to call `fit()` with `X_train` and `y_train` and it'll sequentially fit the transformers which will finally pass the transformed data to the estimator. Then, simply call `predict()` with `X_test` as argument and the pipeline will automatically cross-validate all fitted transformers, and eventually the estimator, on the `X_test` variable. 

Okay, lot's of info - let's break this down. First, let's initialize some transformers and an estimator:

------------
\* [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) is a scikit-learn transformer that does exactly the same as the `standardize()` method you implemented earlier! Strictly speaking, you don't need to cross-validate your standardization step, but it doesn't hurt either. 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
ufs = SelectKBest(score_func=f_classif, k=1000)
pca = PCA(n_components=10)  # we want to reduce the features to 10 components
svc = SVC(kernel='linear')

Now, to initialize a Pipeline-object, we need to give it a list of tuples, in the first entry of each tuple is a name for the step in the pipeline and the second entry of each tuple is the actual transformer/estimator object. Let's do that for our pipeline:

In [ ]:
from sklearn.pipeline import Pipeline

pipeline_to_make = [('preproc', scaler),
                    ('ufs', ufs),
                    ('pca', pca),
                    ('clf', svc)]

my_pipe = Pipeline(pipeline_to_make)

Let's test our pipeline-object (`my_pipe`) on our data. For this example, we'll use a simple hold-out cross-validation scheme (but pipelines are equally valuable in K-fold CV schemes!).

In [ ]:
X_train, y_train = mvp.X[0::2], mvp.y[0::2]
X_test, y_test = mvp.X[1::2], mvp.y[1::2]

my_pipe.fit(X_train, y_train)
predictions = my_pipe.predict(X_test)
acc = np.mean(predictions == y_test)
print("Cross-validated accuracy on test-set: %.3f" % acc)

Cool stuff huh? Quite efficient, I would say! Anyway, let's continue with the last part of this tutorial: permutation testing!

## 5. Permutation testing
While group-level statistics in within-subject decoding analyses are easily implemented (usually with a t-test of subject-wise accuracy scores against chance level), between-subject decoding analyses are trickier. That is because the data-points in between-subject analyses - usually the accuracy scores associated with different folds in K-fold CV - are *not independent* - which is an important assumption of all parametric statistics (such as t-tests). 

As explained in the lecture, dependence between data-points is created because the same samples may be included in different folds. This may not immediately make sense, but imagine the following: suppose we have 100 samples. For some reason, there is one sample that is *suuuuper* noisy. In fact, everytime it is included in the train-set (90%, so 90 trials), it messes up the fit of the model so badly, that it always leads to chance-level predictions on the test-set (on average only 5 samples in the test set are predicted correctly). When this noisy sample is included in the test-set, however, it doesn't mess up the model fit, and on average the test-set is predicted correctly with 80% accuracy. 

As you can see, this super noisy sample actually creates dependence between the accuracies associated with the folds in which it has been part of the train-set: these accuracies are *all* relatively lower than the folds in which this sample was part of the test-set. 

Anyway, long story short: parametric statistics don't work, so let's see how to implement a permutation test!

### 5.1. Getting your *observed* performance score
Before you start with anything related to permutation testing, you first need to get your performance score that you would like to get a p-values for. In between-subject decoding analyses, this is usually the average accuracy (or whatever other metric) across folds. So, when you would implement a 10-fold cross-validation scheme, your observed performance score would be the average of the accuracy-estimates across those 10 folds. You know what? Let's implement exactly that.

This time, however, we won't do any data-drive feature selection, but we'll implement a ROI-based decoding analysis in which we only use the patterns in the left amygdala. This choice for the left amygdala is not theoretically motivated (the amygdala is unlikely to contain any information during a working memory task that is predictive for intelligence ... but you never know...), but is rather a practical concern: permutation testing takes a loooooot of time, and by limiting ourselves to a small subset of features (i.e. voxels in the left amygdala), we substantially speed up the (permutation) analysis.

Anyway, let's implement such an analysis:

In [ ]:
mvp.apply_mask('Left_Amygdala_mask.nii.gz', threshold=50)

In [ ]:
###### Apply amygdala mask
print("Shape of mvp.X: %r" % (mvp.X.shape,))

clf = LogisticRegression()

# We'll pick seven folds
n_folds = 2
skf = StratifiedKFold(n_splits=n_folds)
folds = skf.split(mvp.X, mvp.y)

accuracy = []

for fold in folds:
    
    train_idx, test_idx = fold
    X_train, X_test = mvp.X[train_idx], mvp.X[test_idx]
    y_train, y_test = mvp.y[train_idx], mvp.y[test_idx]
    
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    accuracy.append(np.mean(pred == y_test))
    
observed_acc = np.mean(accuracy)
print("Mean accuracy across folds = %.3f" % observed_acc)

Alright, not bad! 52.9% correct is higher than chance, but is it also *significantly* higher than chance? For that, we need to "simulate" a null-distribution through permutation!

### 5.2. Getting your *permuted* performance distribution
The goal of permutation tests are to create a null-distribution of your observed measure, in this case: (average) classification accuracy. The null-distribution refers to the distribution of classification accuracies that would arise if you would repeat an experiment with noisy data in which there is no effect (i.e. the null-hypothesis is true). 

Thus, we need to somehow repeat the above "experiment" (i.e. the analysis which yielded the observed performance of 52.9%) yet while the null-hypothesis is true (i.e. there is no effect, only noise). Well, as the term "permutation" suggest, we can simply randomly shuffly the train-labels (`y_train`) to generate random labels. Now, if we would fit the classifier on {`X_train`, and `y_train`}, then practically it would fit on noise. In fact, by random shuffling of the train-labels, we simulated the scenario in which (on average) the null-hypothesis would be true.

So, what we need to do in order to create a null-distribution of classification-accuracies is to run our original analysis (in the code cell above) many times (i.e. "repeat the experiment"), yet with random labeling of our train-samples. We expect that the mean of these null-accuracies would center around .5 (assumed chance level), but simply due to random noise, our simulated null-distribution will contain values (sometimes substantially) above and below chance level. 

Anyway, look at the code-cell below. It contains exactly the same analysis as in the above code-block, yet now it is looped 1000 times, and just before fitting (within every fold separately) we shuffle the train-labels to generate a random mapping between `X_train` and `y_train`. After every "experiment" (or simply "permutation") we average the permuted accuracy scores across folds and store them (in the variable `permuted_accuracies`). Now, let's run the code cell below (may take minute):

In [ ]:
n_permutations = 1000

permuted_accuracies = np.zeros(n_permutations)
for i in range(n_permutations):

    folds = skf.split(mvp.X, mvp.y)
    fold_accuracies = np.zeros(n_folds)

    for ii, fold in enumerate(folds):
    
        train_idx, test_idx = fold
        X_train, X_test = mvp.X[train_idx], mvp.X[test_idx]
        y_train, y_test = mvp.y[train_idx], mvp.y[test_idx]
    
        # Here we shuffle the y-train labels!!!
        np.random.shuffle(y_train)
        
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        fold_accuracies[ii] = np.mean(pred == y_test)
    
    permuted_accuracies[i] = np.mean(fold_accuracies)

Okay, we now got our simulated null-distribution values stored in the variable `permuted_accuracies`. Let's look at how the distribution actually looks like:

In [ ]:
plt.title('Permuted null-distribution')
plt.hist(permuted_accuracies, bins=25)
plt.xlabel('Average accuracy across folds')
plt.ylabel('Frequency')
plt.show()

Now, remember the 'formula' for the p-value in a permutation test? Given *N* permutations, it's the following (in pseudo-notation):

\begin{align}
p = \frac{1}{N}\sum_{i=1}^{N}{permuted_{i} > observed}
\end{align}

Put differently: the p-value expresses the proportion of permutation-values are higher than the observed value. Graphically, we can visualize the p-value as the area of the null-distribution right of the dotted red line: 

In [ ]:
plt.title('Permuted null-distribution')
plt.hist(permuted_accuracies, bins=25)
plt.xlabel('Average accuracy across folds')
plt.ylabel('Frequency')
plt.axvline(observed_acc, c='r', linestyle='dashed', linewidth=3)
plt.legend(['Observed accuracy'], loc=2, frameon=False)
plt.show()

<div class='alert alert-danger'>
**Assignment 3** (1 point): calculate the p-value corresponding to the observed accuracy given our permuted null-distribution. Also, interpret the result: what does this tell us about our research question? 
</div>

In [ ]:
# calculate p-value